In [1]:
pip install pdfminer.six scikit-learn nltk beautifulsoup4 requests


   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/5.6 MB 1.6 MB/s eta 0:00:04
   ----------- ---------------------------- 1.6/5.6 MB 2.2 MB/s eta 0:00:02
   ---------------- ----------------------- 2.4/5.6 MB 2.7 MB/s eta 0:00:02
   -------------------- ------------------- 2.9/5.6 MB 2.8 MB/s eta 0:00:01
   ------------------------ --------------- 3.4/5.6 MB 2.9 MB/s eta 0:00:01
   --------------------------- ------------ 3.9/5.6 MB 2.6 MB/s eta 0:00:01
   ----------------------------- ---------- 4.2/5.6 MB 2.5 MB/s eta 0:00:01
   --------------------------------- ------ 4.7/5.6 MB 2.4 MB/s eta 0:00:01
   ----------------------------------- ---- 5.0/5.6 MB 2.4 MB/s eta 0:00:01
   -------------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install nltk pdfminer.six beautifulsoup4 requests scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import re
import time
import requests
import nltk
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pdfminer.high_level import extract_text
from nltk.tokenize import sent_tokenize

# Ensure punkt is downloaded
nltk.download('punkt')

# --------------------------
# 1. Extract Text from PDF or TXT
# --------------------------
def extract_text_from_file(file_path):
    try:
        if file_path.endswith(".pdf"):
            return extract_text(file_path)
        elif file_path.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8") as f:
                return f.read()
        else:
            raise ValueError("Only PDF or TXT files are supported.")
    except Exception as e:
        print(f"❌ Failed to extract text: {e}")
        return ""

# --------------------------
# 2. Clean and Split Text into Sentences
# --------------------------
def preprocess_text(text):
    try:
        sentences = sent_tokenize(text)
        return [s.strip() for s in sentences if len(s.strip()) > 30]
    except Exception as e:
        print(f"❌ Tokenization error: {e}")
        return []

# --------------------------
# 3. Search Sentence on Google
# --------------------------
def search_google_snippet(query):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    params = {
        "q": f'"{query}"',
        "hl": "en"
    }
    try:
        response = requests.get("https://www.google.com/search", params=params, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        snippets = soup.select("div.BNeawe.s3v9rd.AP7Wnd")
        return " ".join([s.get_text() for s in snippets])
    except Exception as e:
        print(f"❌ Google search failed: {e}")
        return ""

# --------------------------
# 4. Compare Sentences Using TF-IDF Similarity
# --------------------------
def check_plagiarism(sentences, threshold=0.75, delay=2):
    plagiarized = []
    for i, sentence in enumerate(sentences):
        print(f"🔍 Checking sentence {i+1}/{len(sentences)}...")
        snippet = search_google_snippet(sentence)
        if snippet:
            vec = TfidfVectorizer().fit_transform([sentence, snippet])
            score = cosine_similarity(vec[0], vec[1])[0][0]
            if score > threshold:
                plagiarized.append((sentence, round(score, 3)))
        time.sleep(delay)  # Prevent getting blocked by Google
    return plagiarized

# --------------------------
# 5. Run the Plagiarism Checker
# --------------------------
def run_plagiarism_checker(file_path):
    print(f"📄 Extracting from {file_path} ...")
    raw_text = extract_text_from_file(file_path)
    
    if not raw_text:
        print("⚠️ No text extracted.")
        return
    
    sentences = preprocess_text(raw_text)
    
    if not sentences:
        print("⚠️ No valid sentences found.")
        return
    
    results = check_plagiarism(sentences)

    print("\n🧾 Plagiarized Sentences Found:\n")
    if not results:
        print("✅ No plagiarism detected.")
    else:
        for sent, score in results:
            print(f"🔴 Score: {score} → {sent[:200]}...")

# --------------------------
# 📌 USAGE
# --------------------------
your_paper_path = r"C:\Users\sagni\Downloads\Band Selection\paper\IIIT Paper.pdf"
run_plagiarism_checker(your_paper_path)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sagni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


📄 Extracting from C:\Users\sagni\Downloads\Band Selection\paper\IIIT Paper.pdf ...
❌ Tokenization error: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\sagni/nltk_data'
    - 'C:\\Users\\sagni\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'C:\\Users\\sagni\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'C:\\Users\\sagni\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\sagni\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************

⚠️ No valid sentences found.
